In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

class LoyaltyPointsEDA:
    def __init__(self, df):
        """Initialize with transaction dataframe"""
        self.df = df.copy()
        self.cleaned_df = None
        self.customer_summary = None
        
    def initial_exploration(self):
        """Perform initial data exploration"""
        print("="*80)
        print("INITIAL DATA EXPLORATION")
        print("="*80)
        
        print(f"\nDataset Shape: {self.df.shape}")
        print(f"\nColumns: {list(self.df.columns)}")
        
        print("\n" + "-"*80)
        print("Data Types:")
        print("-"*80)
        print(self.df.dtypes)
        
        print("\n" + "-"*80)
        print("First Few Rows:")
        print("-"*80)
        print(self.df.head())
        
        print("\n" + "-"*80)
        print("Statistical Summary:")
        print("-"*80)
        print(self.df.describe())
        
        print("\n" + "-"*80)
        print("Missing Values:")
        print("-"*80)
        missing = self.df.isnull().sum()
        missing_pct = (missing / len(self.df)) * 100
        missing_df = pd.DataFrame({
            'Missing_Count': missing,
            'Percentage': missing_pct
        })
        print(missing_df[missing_df['Missing_Count'] > 0])
        
        print("\n" + "-"*80)
        print("Duplicate Rows:")
        print("-"*80)
        print(f"Number of duplicates: {self.df.duplicated().sum()}")
        
    def clean_data(self):
        """Clean and preprocess the data"""
        print("\n" + "="*80)
        print("DATA CLEANING")
        print("="*80)
        
        df_clean = self.df.copy()
        
        print("\n1. Handling Missing Values...")
        for col in df_clean.columns:
            if df_clean[col].isnull().sum() > 0:
                if df_clean[col].dtype in ['float64', 'int64']:
                    df_clean[col].fillna(df_clean[col].median(), inplace=True)
                    print(f"   - Filled {col} with median")
                else:
                    df_clean[col].fillna('Unknown', inplace=True)
                    print(f"   - Filled {col} with 'Unknown'")
        
        print("\n2. Removing Duplicates...")
        before = len(df_clean)
        df_clean.drop_duplicates(inplace=True)
        after = len(df_clean)
        print(f"   - Removed {before - after} duplicate rows")
        
        print("\n3. Converting Date Columns...")
        date_cols = [col for col in df_clean.columns if 'date' in col.lower() or 'time' in col.lower()]
        for col in date_cols:
            try:
                df_clean[col] = pd.to_datetime(df_clean[col])
                print(f"   - Converted {col} to datetime")
            except:
                print(f"   - Could not convert {col}")
        
        print("\n4. Detecting Outliers (IQR method)...")
        numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            Q1 = df_clean[col].quantile(0.25)
            Q3 = df_clean[col].quantile(0.75)
            IQR = Q3 - Q1
            outliers = ((df_clean[col] < (Q1 - 1.5 * IQR)) | (df_clean[col] > (Q3 + 1.5 * IQR))).sum()
            if outliers > 0:
                print(f"   - {col}: {outliers} outliers detected ({outliers/len(df_clean)*100:.2f}%)")
        
        self.cleaned_df = df_clean
        print("\nData cleaning completed!")
        return df_clean
    
    def visualize_data(self):
        """Create visualizations for EDA"""
        if self.cleaned_df is None:
            print("Please run clean_data() first!")
            return
        
        print("\n" + "="*80)
        print("DATA VISUALIZATIONS")
        print("="*80)
        
        df = self.cleaned_df
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        
        # Distribution plots
        if len(numeric_cols) > 0:
            fig, axes = plt.subplots(min(len(numeric_cols), 3), 1, figsize=(12, 4*min(len(numeric_cols), 3)))
            if len(numeric_cols) == 1:
                axes = [axes]
            
            for i, col in enumerate(numeric_cols[:3]):
                df[col].hist(bins=30, ax=axes[i], edgecolor='black')
                axes[i].set_title(f'Distribution of {col}')
                axes[i].set_xlabel(col)
                axes[i].set_ylabel('Frequency')
            
            plt.tight_layout()
            plt.show()
        
        if len(numeric_cols) > 1:
            plt.figure(figsize=(10, 8))
            sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', center=0, fmt='.2f')
            plt.title('Correlation Heatmap')
            plt.tight_layout()
            plt.show()
    
    def calculate_loyalty_points(self, transaction_amt_col, points_rate=0.1):
        """
        Calculate accrued points for current transactions
        
        Parameters:
        -----------
        transaction_amt_col : str
            Column name containing transaction amount
        points_rate : float
            Points earned per dollar (default: 0.1 = 10 points per $100)
        """
        if self.cleaned_df is None:
            print("Please run clean_data() first!")
            return
        
        print("\n" + "="*80)
        print("CALCULATING LOYALTY POINTS")
        print("="*80)
        
        df = self.cleaned_df.copy()
        
        df['accrued_points'] = df[transaction_amt_col] * points_rate
        df['accrued_points'] = df['accrued_points'].round(2)
        
        print(f"\nPoints Rate: {points_rate} points per dollar")
        print(f"Total Points Accrued: {df['accrued_points'].sum():,.2f}")
        print(f"Average Points per Transaction: {df['accrued_points'].mean():,.2f}")
        
        self.cleaned_df = df
        return df
    
    def update_customer_balance(self, customer_id_col, initial_balance=0):
        """
        Update customer's total point balance
        
        Parameters:
        -----------
        customer_id_col : str
            Column name containing customer ID
        initial_balance : float
            Initial balance for new customers
        """
        if self.cleaned_df is None or 'accrued_points' not in self.cleaned_df.columns:
            print("Please run calculate_loyalty_points() first!")
            return
        
        print("\n" + "="*80)
        print("UPDATING CUSTOMER POINT BALANCES")
        print("="*80)
        
        df = self.cleaned_df.copy()
        
        customer_points = df.groupby(customer_id_col).agg({
            'accrued_points': 'sum'
        }).reset_index()
        
        customer_points.rename(columns={'accrued_points': 'total_point_balance'}, inplace=True)
        customer_points['total_point_balance'] += initial_balance
        
        df = df.merge(customer_points, on=customer_id_col, how='left')
        
        print(f"\nTotal Customers: {df[customer_id_col].nunique()}")
        print(f"Total Points Distributed: {customer_points['total_point_balance'].sum():,.2f}")
        print(f"Average Balance per Customer: {customer_points['total_point_balance'].mean():,.2f}")
        print(f"Top Customer Balance: {customer_points['total_point_balance'].max():,.2f}")
        
        self.cleaned_df = df
        self.customer_summary = customer_points
        return df, customer_points
    
    def calculate_rfm(self, customer_id_col, date_col, transaction_amt_col, reference_date=None):
        """
        Calculate RFM (Recency, Frequency, Monetary) metrics
        
        Parameters:
        -----------
        customer_id_col : str
            Column name containing customer ID
        date_col : str
            Column name containing transaction date
        transaction_amt_col : str
            Column name containing transaction amount
        reference_date : datetime
            Reference date for recency calculation (default: max date in data)
        """
        if self.cleaned_df is None:
            print("Please run clean_data() first!")
            return
        
        print("\n" + "="*80)
        print("CALCULATING RFM METRICS")
        print("="*80)
        
        df = self.cleaned_df.copy()
        
        if reference_date is None:
            reference_date = df[date_col].max()
        
        rfm = df.groupby(customer_id_col).agg({
            date_col: lambda x: (reference_date - x.max()).days,  # Recency
            customer_id_col: 'count',
            transaction_amt_col: 'sum'
        }).reset_index()
        
        rfm.columns = [customer_id_col, 'recency', 'frequency', 'monetary']
        
        rfm['R_score'] = pd.qcut(rfm['recency'], 5, labels=[5,4,3,2,1], duplicates='drop')
        rfm['F_score'] = pd.qcut(rfm['frequency'].rank(method='first'), 5, labels=[1,2,3,4,5], duplicates='drop')
        rfm['M_score'] = pd.qcut(rfm['monetary'], 5, labels=[1,2,3,4,5], duplicates='drop')
        
        rfm['R_score'] = rfm['R_score'].astype(int)
        rfm['F_score'] = rfm['F_score'].astype(int)
        rfm['M_score'] = rfm['M_score'].astype(int)
        
        rfm['RFM_score'] = rfm['R_score'] + rfm['F_score'] + rfm['M_score']
        
        rfm['customer_segment'] = rfm['RFM_score'].apply(self._segment_customer)
        
        self.cleaned_df = df.merge(rfm[[customer_id_col, 'recency', 'frequency', 'monetary', 
                                         'R_score', 'F_score', 'M_score', 'RFM_score', 
                                         'customer_segment']], on=customer_id_col, how='left')
        
        print(f"\nRFM Analysis Summary:")
        print(f"Reference Date: {reference_date.strftime('%Y-%m-%d')}")
        print(f"\nRecency (days since last purchase):")
        print(f"  - Mean: {rfm['recency'].mean():.1f} days")
        print(f"  - Median: {rfm['recency'].median():.1f} days")
        print(f"\nFrequency (number of purchases):")
        print(f"  - Mean: {rfm['frequency'].mean():.1f} transactions")
        print(f"  - Median: {rfm['frequency'].median():.1f} transactions")
        print(f"\nMonetary (total spend):")
        print(f"  - Mean: ${rfm['monetary'].mean():,.2f}")
        print(f"  - Median: ${rfm['monetary'].median():,.2f}")
        
        print("\n" + "-"*80)
        print("Customer Segmentation:")
        print("-"*80)
        print(rfm['customer_segment'].value_counts().sort_index())
        
        return self.cleaned_df, rfm
    
    def _segment_customer(self, score):
        """Segment customers based on RFM score"""
        if score >= 13:
            return 'Champions'
        elif score >= 10:
            return 'Loyal Customers'
        elif score >= 7:
            return 'Potential Loyalists'
        elif score >= 5:
            return 'At Risk'
        else:
            return 'Lost'
    
    def generate_report(self):
        """Generate comprehensive summary report"""
        if self.cleaned_df is None:
            print("Please complete the analysis pipeline first!")
            return
        
        print("\n" + "="*80)
        print("COMPREHENSIVE LOYALTY PROGRAM REPORT")
        print("="*80)
        
        df = self.cleaned_df
        
        print("\nDATASET OVERVIEW")
        print(f"  - Total Transactions: {len(df):,}")
        print(f"  - Date Range: {df[df.select_dtypes(include=['datetime64']).columns[0]].min().strftime('%Y-%m-%d')} to {df[df.select_dtypes(include=['datetime64']).columns[0]].max().strftime('%Y-%m-%d')}")
        
        if 'accrued_points' in df.columns:
            print("\nPOINTS SUMMARY")
            print(f"  - Total Points Accrued: {df['accrued_points'].sum():,.2f}")
            print(f"  - Average Points per Transaction: {df['accrued_points'].mean():,.2f}")
        
        if 'total_point_balance' in df.columns:
            print("\nCUSTOMER BALANCES")
            print(f"  - Total Customers: {df[df.columns[0]].nunique():,}")
            print(f"  - Average Balance: {df.groupby(df.columns[0])['total_point_balance'].first().mean():,.2f}")
        
        if 'customer_segment' in df.columns:
            print("\nCUSTOMER SEGMENTS")
            segments = df.groupby(df.columns[0])['customer_segment'].first().value_counts()
            for segment, count in segments.items():
                print(f"  - {segment}: {count} ({count/len(segments)*100:.1f}%)")
        
        print("\n" + "="*80)


def run_loyalty_analysis(file_path, 
                         customer_id_col='customer_id',
                         date_col='transaction_date', 
                         transaction_amt_col='amount',
                         points_rate=0.1,
                         initial_balance=0,
                         output_folder='output'):
    """
    Complete loyalty points analysis pipeline
    
    Parameters:
    -----------
    file_path : str
        Path to CSV file containing transaction data
    customer_id_col : str
        Column name for customer ID
    date_col : str
        Column name for transaction date
    transaction_amt_col : str
        Column name for transaction amount
    points_rate : float
        Points earned per dollar (default: 0.1)
    initial_balance : float
        Initial balance for customers
    output_folder : str
        Folder to save output files
    
    Returns:
    --------
    df_final : DataFrame
        Cleaned data with all calculations
    rfm_df : DataFrame
        RFM analysis summary
    customer_summary : DataFrame
        Customer point balances
    """
    import os
    
    os.makedirs(output_folder, exist_ok=True)
    
    print("LOYALTY POINTS ANALYSIS PIPELINE")
    
    print("\nLoading data from:", file_path)
    try:
        df = pd.read_csv(file_path)
        print(f"Successfully loaded {len(df):,} rows and {len(df.columns)} columns")
    except Exception as e:
        print(f"Error loading file: {e}")
        return None, None, None
    
    print("\n🔧 Initializing analysis...")
    eda = LoyaltyPointsEDA(df)
    
    eda.initial_exploration()
    
    df_clean = eda.clean_data()
    
    try:
        eda.visualize_data()
    except:
        print("Skipping visualizations")
    
    eda.calculate_loyalty_points(
        transaction_amt_col=transaction_amt_col, 
        points_rate=points_rate
    )
    
    df_with_balance, customer_summary = eda.update_customer_balance(
        customer_id_col=customer_id_col,
        initial_balance=initial_balance
    )
    
    df_final, rfm_df = eda.calculate_rfm(
        customer_id_col=customer_id_col,
        date_col=date_col,
        transaction_amt_col=transaction_amt_col
    )
    
    eda.generate_report()
    
    print("\n" + "="*80)
    print("EXPORTING RESULTS")
    print("="*80)
    
    output_files = {
        'cleaned_data': os.path.join(output_folder, 'cleaned_loyalty_data.csv'),
        'rfm_analysis': os.path.join(output_folder, 'rfm_analysis.csv'),
        'customer_summary': os.path.join(output_folder, 'customer_point_balances.csv')
    }
    
    try:
        df_final.to_csv(output_files['cleaned_data'], index=False)
        print(f"Cleaned data saved to: {output_files['cleaned_data']}")
        
        rfm_df.to_csv(output_files['rfm_analysis'], index=False)
        print(f"RFM analysis saved to: {output_files['rfm_analysis']}")
        
        customer_summary.to_csv(output_files['customer_summary'], index=False)
        print(f"Customer balances saved to: {output_files['customer_summary']}")
    except Exception as e:
        print(f"Error saving files: {e}")
    
    print("ANALYSIS COMPLETE!")
    
    return df_final, rfm_df, customer_summary


if __name__ == "__main__":
    df_final, rfm_df, customer_summary = run_loyalty_analysis(
        file_path='transactions.csv'
    )

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)


def perform_eda(file_path):
    """
    Perform comprehensive Exploratory Data Analysis on a dataset
    
    Parameters:
    -----------
    file_path : str
        Path to the CSV file
    
    Returns:
    --------
    df : DataFrame
        Loaded dataframe
    """
    
    print("="*80)
    print("EXPLORATORY DATA ANALYSIS (EDA)")
    print("="*80)
    
    print("\n📁 STEP 1: LOADING DATA")
    print("-"*80)
    try:
        df = pd.read_csv(file_path)
        print(f"✅ Successfully loaded: {file_path}")
        print(f"   Rows: {df.shape[0]:,}")
        print(f"   Columns: {df.shape[1]}")
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        return None
    
    print("\n" + "="*80)
    print("📊 STEP 2: BASIC INFORMATION")
    print("="*80)
    
    print("\n🔹 Column Names:")
    for i, col in enumerate(df.columns, 1):
        print(f"   {i}. {col}")
    
    print("\n🔹 Data Types:")
    print(df.dtypes)
    
    print("\n🔹 Dataset Shape:")
    print(f"   Rows: {df.shape[0]:,}")
    print(f"   Columns: {df.shape[1]}")
    print(f"   Total Cells: {df.shape[0] * df.shape[1]:,}")
    
    print("\n🔹 Memory Usage:")
    memory = df.memory_usage(deep=True).sum() / 1024**2
    print(f"   {memory:.2f} MB")
    
    print("\n" + "="*80)
    print("👀 STEP 3: PREVIEW DATA")
    print("="*80)
    
    print("\n🔹 First 5 Rows:")
    print(df.head())
    
    print("\n🔹 Last 5 Rows:")
    print(df.tail())
    
    print("\n" + "="*80)
    print("🔍 STEP 4: MISSING VALUES ANALYSIS")
    print("="*80)
    
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Column': missing.index,
        'Missing_Count': missing.values,
        'Percentage': missing_pct.values
    })
    missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)
    
    if len(missing_df) > 0:
        print("\nColumns with Missing Values:")
        print(missing_df.to_string(index=False))
        print(f"\nTotal missing values: {missing.sum():,}")
    else:
        print("\nNo missing values found!")
    
    print("\n" + "="*80)
    print("STEP 5: DUPLICATE ROWS")
    print("="*80)
    
    duplicates = df.duplicated().sum()
    print(f"\nDuplicate rows: {duplicates:,} ({duplicates/len(df)*100:.2f}%)")
    
    if duplicates > 0:
        print("Consider removing duplicates during data cleaning")
    else:
        print("No duplicates found!")
    
    print("\n" + "="*80)
    print("STEP 6: STATISTICAL SUMMARY")
    print("="*80)
    
    print("\nNumeric Columns:")
    print(df.describe())
    
    print("\nCategorical Columns:")
    categorical_cols = df.select_dtypes(include=['object']).columns
    if len(categorical_cols) > 0:
        for col in categorical_cols:
            print(f"\n   {col}:")
            print(f"      Unique values: {df[col].nunique()}")
            print(f"      Most common: {df[col].mode()[0] if len(df[col].mode()) > 0 else 'N/A'}")
            if df[col].nunique() <= 10:
                print(f"      Value counts:")
                print(df[col].value_counts().head(10).to_string())
    else:
        print("   No categorical columns found")
    
    print("\n" + "="*80)
    print("STEP 7: DATA TYPES BREAKDOWN")
    print("="*80)
    
    print("\n🔹 Column Type Distribution:")
    dtype_counts = df.dtypes.value_counts()
    for dtype, count in dtype_counts.items():
        print(f"   {dtype}: {count} columns")
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    datetime_cols = df.select_dtypes(include=['datetime64']).columns.tolist()
    
    print(f"\n   Numeric columns ({len(numeric_cols)}): {numeric_cols}")
    print(f"   Categorical columns ({len(categorical_cols)}): {categorical_cols}")
    print(f"   Datetime columns ({len(datetime_cols)}): {datetime_cols}")
    
    print("\n" + "="*80)
    print("STEP 8: UNIQUE VALUES PER COLUMN")
    print("="*80)
    
    print("\n")
    unique_df = pd.DataFrame({
        'Column': df.columns,
        'Unique_Values': [df[col].nunique() for col in df.columns],
        'Unique_Percentage': [df[col].nunique()/len(df)*100 for col in df.columns]
    }).sort_values('Unique_Values', ascending=False)
    
    print(unique_df.to_string(index=False))
    
    print("STEP 9: OUTLIER DETECTION")
    
    print("\nOutliers (IQR Method):")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    outlier_summary = []
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outlier_count = len(outliers)
        outlier_pct = (outlier_count / len(df)) * 100
        
        if outlier_count > 0:
            outlier_summary.append({
                'Column': col,
                'Outliers': outlier_count,
                'Percentage': f"{outlier_pct:.2f}%",
                'Lower_Bound': f"{lower_bound:.2f}",
                'Upper_Bound': f"{upper_bound:.2f}"
            })
    
    if outlier_summary:
        outlier_df = pd.DataFrame(outlier_summary)
        print(outlier_df.to_string(index=False))
    else:
        print("   ✅ No outliers detected!")
    
    print("\n" + "="*80)
    print("🔗 STEP 10: CORRELATION ANALYSIS")
    print("="*80)
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 1:
        print("\n🔹 Correlation Matrix:")
        corr_matrix = df[numeric_cols].corr()
        print(corr_matrix)
        
        print("\n🔹 High Correlations (|r| > 0.7):")
        high_corr = []
        for i in range(len(corr_matrix.columns)):
            for j in range(i+1, len(corr_matrix.columns)):
                if abs(corr_matrix.iloc[i, j]) > 0.7:
                    high_corr.append({
                        'Feature_1': corr_matrix.columns[i],
                        'Feature_2': corr_matrix.columns[j],
                        'Correlation': f"{corr_matrix.iloc[i, j]:.3f}"
                    })
        
        if high_corr:
            high_corr_df = pd.DataFrame(high_corr)
            print(high_corr_df.to_string(index=False))
        else:
            print("   No high correlations found")
    else:
        print(" Not enough numeric columns for correlation analysis")
    
    print("\n" + "="*80)
    print("STEP 11: GENERATING VISUALIZATIONS")
    print("="*80)
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numeric_cols) > 0:
        n_cols = min(len(numeric_cols), 4)
        fig, axes = plt.subplots(n_cols, 1, figsize=(12, 4*n_cols))
        if n_cols == 1:
            axes = [axes]
        
        for i, col in enumerate(numeric_cols[:4]):
            df[col].hist(bins=30, ax=axes[i], edgecolor='black', alpha=0.7)
            axes[i].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
            axes[i].set_xlabel(col)
            axes[i].set_ylabel('Frequency')
            axes[i].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        print("Distribution plots generated")
    
    # Correlation heatmap
    if len(numeric_cols) > 1:
        plt.figure(figsize=(10, 8))
        sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', 
                    center=0, fmt='.2f', square=True, linewidths=1)
        plt.title('Correlation Heatmap', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        print("Correlation heatmap generated")
    
    if df.isnull().sum().sum() > 0:
        plt.figure(figsize=(10, 6))
        missing_data = df.isnull().sum()[df.isnull().sum() > 0].sort_values(ascending=False)
        missing_data.plot(kind='bar', color='coral', edgecolor='black')
        plt.title('Missing Values by Column', fontsize=14, fontweight='bold')
        plt.xlabel('Columns')
        plt.ylabel('Missing Count')
        plt.xticks(rotation=45, ha='right')
        plt.grid(True, alpha=0.3, axis='y')
        plt.tight_layout()
        plt.show()
        print("Missing values chart generated")
    
    
    print("\n" + "="*80)
    print("STEP 12: EDA SUMMARY")
    print("="*80)
    
    print(f"""
EDA COMPLETED SUCCESSFULLY!

Key Findings:
   • Total Rows: {df.shape[0]:,}
   • Total Columns: {df.shape[1]}
   • Missing Values: {df.isnull().sum().sum():,}
   • Duplicate Rows: {duplicates:,}
   • Numeric Columns: {len(df.select_dtypes(include=[np.number]).columns)}
   • Categorical Columns: {len(df.select_dtypes(include=['object']).columns)}
   • Memory Usage: {memory:.2f} MB

Next Steps:
   1. Handle missing values
   2. Remove duplicates if necessary
   3. Handle outliers
   4. Proceed with data cleaning and feature engineering
    """)
    
    return df


if __name__ == "__main__":
    file_path = '/kaggle/input/retail/retail_data.csv'
    
    df = perform_eda(file_path)
    
    if df is not None:
        print("\n✅ DataFrame is ready for further analysis!")

****Final pipeline**************

**************Pipeline dividing into 5 tables**********************************

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime


def parse_mixed_date(series: pd.Series) -> pd.Series:
    s = series.astype(str)
    res = pd.Series(pd.NaT, index=s.index, dtype='datetime64[ns]')
    mask_slash = s.str.contains("/")
    mask_dash = s.str.contains("-")

    res.loc[mask_slash] = pd.to_datetime(s.loc[mask_slash],
        format="%m/%d/%Y", errors="coerce")

    res.loc[mask_dash] = pd.to_datetime(s.loc[mask_dash],
        format="%m-%d-%y", errors="coerce")

    return res


def safe_int_str(series: pd.Series) -> pd.Series:
    return series.astype("Int64").astype(str)


CSV_PATH = "/kaggle/input/good-dataset/good_data.csv"
DB_URL = "sqlite:///retail.db"

print("🔹 Loading CSV...")
raw = pd.read_csv(CSV_PATH)

raw["tx_date"] = parse_mixed_date(raw["Date"])
raw["customer_id"] = safe_int_str(raw["Customer_ID"])
raw["transaction_id"] = safe_int_str(raw["Transaction_ID"])


def build_customer_master(df):
    g = df.dropna(subset=["customer_id"]).groupby("customer_id")

    cm = g.agg({
        "Name":"first","Email":"first","Phone":"first",
        "Address":"first","City":"first","State":"first",
        "Zipcode":"first","Country":"first","Age":"first",
        "Gender":"first","Income":"first","Customer_Segment":"first",
        "tx_date":["min","max"]
    }).reset_index()

    cm.columns = [
        "customer_id","name","email","phone","address","city","state","zipcode",
        "country","age","gender","income","customer_segment",
        "customer_since","last_purchase_date"
    ]

    cm["is_loyalty_member"] = True
    cm["total_loyalty_points"] = 0
    cm["bonus_points"] = 0
    cm["last_points_update"] = pd.NaT

    cm["zipcode"] = cm["zipcode"].astype("Int64").astype(str)
    cm["age"] = cm["age"].astype("Int64")
    cm["income"] = pd.to_numeric(cm["income"], errors="coerce")

    # CLEAN duplicated keys
    cm = cm.drop_duplicates(subset=["customer_id"])

    return cm


def build_product_master(df):
    pm = df[["Product_Category","Product_Brand","Product_Type","products"]].drop_duplicates()
    pm = pm.rename(columns={
        "Product_Category":"product_category",
        "Product_Brand":"product_brand",
        "Product_Type":"product_type",
        "products":"product_full_name"
    })

    pm = pm.reset_index(drop=True)
    pm["product_key"] = pm.index.to_series().add(1).map(lambda x: f"P{x:06d}")
    pm["is_active"] = True

    return pm[[
        "product_key","product_category","product_brand","product_type",
        "product_full_name","is_active"
    ]]


def build_sales_transactions(df):
    st = df.copy()
    st["date"] = st["tx_date"]
    st["year"] = st["date"].dt.year
    st["month"] = st["date"].dt.month

    month_map = {
        "January":1,"February":2,"March":3,"April":4,"May":5,"June":6,
        "July":7,"August":8,"September":9,"October":10,"November":11,"December":12
    }
    nat_mask = st["date"].isna()
    st.loc[nat_mask,"year"] = st.loc[nat_mask,"Year"].astype("Int64")
    st.loc[nat_mask,"month"] = st.loc[nat_mask,"Month"].map(month_map)

    st["ingestion_timestamp"] = datetime.utcnow()
    st["data_quality_flag"] = "PASS"
    st["reject_reason"] = pd.NA

    st_df = st.rename(columns={
        "Time":"time",
        "Total_Purchases":"total_purchases",
        "Amount":"amount",
        "Total_Amount":"total_amount",
        "Product_Category":"product_category",
        "Product_Brand":"product_brand",
        "Product_Type":"product_type",
        "Shipping_Method":"shipping_method",
        "Payment_Method":"payment_method",
        "Order_Status":"order_status",
        "Ratings":"ratings",
        "Feedback":"feedback"
    })

    st_df = st_df[[
        "transaction_id","customer_id","date","year","month","time",
        "total_purchases","amount","total_amount","product_category",
        "product_brand","product_type","shipping_method","payment_method",
        "order_status","ratings","feedback","ingestion_timestamp",
        "data_quality_flag","reject_reason"
    ]]

    # CLEAN null + duplicate transaction IDs
    st_df = st_df.dropna(subset=["transaction_id"])
    st_df = st_df[st_df["transaction_id"] != "<NA>"]
    st_df = st_df.drop_duplicates(subset=["transaction_id"])

    return st_df


def build_customer_analytics(st_df):
    snapshot_date = st_df["date"].max()
    g = st_df.groupby("customer_id")

    base = g.agg({
        "date":"max","transaction_id":"nunique","total_amount":"sum",
        "product_type":"nunique","ratings":"mean"
    }).reset_index()

    base = base.rename(columns={
        "date":"last_purchase_date","transaction_id":"frequency",
        "total_amount":"monetary","product_type":"product_diversity",
        "ratings":"avg_rating"
    })

    base["recency"] = (snapshot_date - base["last_purchase_date"]).dt.days
    base["avg_rating"] = base["avg_rating"].fillna(0)
    base["monetary"] = base["monetary"].fillna(0)
    base["recency"] = base["recency"].fillna(base["recency"].max())

    base["rfm_score"] = 1
    base["segment"] = "Medium"
    base["clv_score"] = (
        base["monetary"] * base["frequency"] / (base["recency"] + 1)
    )

    base["snapshot_date"] = snapshot_date

    return base[[
        "customer_id","recency","frequency","monetary",
        "rfm_score","segment","product_diversity","avg_rating",
        "clv_score","snapshot_date"
    ]]


def build_loyalty_transactions(st_df):
    lt = st_df[[
        "transaction_id","customer_id","date","total_amount"
    ]].copy()

    lt["points_earned"] = lt["total_amount"].fillna(0).floordiv(10).astype(int)
    lt["points_redeemed"] = 0
    lt["bonus_points"] = 0

    lt = lt.sort_values(["customer_id","date","transaction_id"])
    lt["balance_after"] = lt.groupby("customer_id")["points_earned"].cumsum()

    lt["event_date"] = lt["date"]
    lt["event_type"] = "EARN"

    lt = lt.reset_index(drop=True)
    lt["loyalty_txn_id"] = lt.index.to_series().add(1).map(lambda x: f"L{x:07d}")

    return lt[[
        "loyalty_txn_id","customer_id","transaction_id","points_earned",
        "points_redeemed","bonus_points","balance_after","event_date",
        "event_type"
    ]]

print("🔹 Building normalized tables...")

customer_master_df = build_customer_master(raw)
product_master_df = build_product_master(raw)
sales_transactions_df = build_sales_transactions(raw)
customer_analytics_df = build_customer_analytics(sales_transactions_df)
loyalty_transactions_df = build_loyalty_transactions(sales_transactions_df)


engine = create_engine(DB_URL)

DDL = [
    "DROP TABLE IF EXISTS loyalty_transactions;",
    "DROP TABLE IF EXISTS customer_analytics;",
    "DROP TABLE IF EXISTS sales_transactions;",
    "DROP TABLE IF EXISTS product_master;",
    "DROP TABLE IF EXISTS customer_master;",

    """
    CREATE TABLE customer_master (
        customer_id VARCHAR PRIMARY KEY,
        name VARCHAR,
        email VARCHAR,
        phone VARCHAR,
        address VARCHAR,
        city VARCHAR,
        state VARCHAR,
        zipcode VARCHAR,
        country VARCHAR,
        age INT,
        gender VARCHAR,
        income DECIMAL,
        customer_segment VARCHAR,
        is_loyalty_member BOOLEAN,
        customer_since DATE,
        total_loyalty_points INT,
        bonus_points INT,
        last_points_update DATE,
        last_purchase_date DATE
    );
    """,

    """
    CREATE TABLE product_master (
        product_key VARCHAR PRIMARY KEY,
        product_category VARCHAR,
        product_brand VARCHAR,
        product_type VARCHAR,
        product_full_name VARCHAR,
        is_active BOOLEAN
    );
    """,

    """
    CREATE TABLE sales_transactions (
        transaction_id VARCHAR PRIMARY KEY,
        customer_id VARCHAR,
        date DATE,
        year INT,
        month INT,
        time VARCHAR,
        total_purchases INT,
        amount DECIMAL,
        total_amount DECIMAL,
        product_category VARCHAR,
        product_brand VARCHAR,
        product_type VARCHAR,
        shipping_method VARCHAR,
        payment_method VARCHAR,
        order_status VARCHAR,
        ratings INT,
        feedback TEXT,
        ingestion_timestamp TIMESTAMP,
        data_quality_flag VARCHAR,
        reject_reason TEXT,
        FOREIGN KEY(customer_id) REFERENCES	customer_master(customer_id)
    );
    """,

    """
    CREATE TABLE customer_analytics (
        customer_id VARCHAR PRIMARY KEY,
        recency INT,
        frequency INT,
        monetary DECIMAL,
        rfm_score INT,
        segment VARCHAR,
        product_diversity INT,
        avg_rating DECIMAL,
        clv_score DECIMAL,
        snapshot_date DATE,
        FOREIGN KEY(customer_id) REFERENCES customer_master(customer_id)
    );
    """,

    """
    CREATE TABLE loyalty_transactions (
        loyalty_txn_id VARCHAR PRIMARY KEY,
        customer_id VARCHAR,
        transaction_id VARCHAR,
        points_earned INT,
        points_redeemed INT,
        bonus_points INT,
        balance_after INT,
        event_date DATE,
        event_type VARCHAR,
        FOREIGN KEY(customer_id) REFERENCES customer_master(customer_id),
        FOREIGN KEY(transaction_id) REFERENCES sales_transactions(transaction_id)
    );
    """
]

print("🔹 Creating database schema...")
with engine.begin() as conn:
    conn.exec_driver_sql("PRAGMA foreign_keys = OFF;")  
    for stmt in DDL:
        conn.exec_driver_sql(stmt)
    conn.exec_driver_sql("PRAGMA foreign_keys = ON;")    


print("🔹 Loading data into DB...")

customer_master_df.to_sql("customer_master", engine, if_exists="append", index=False)
product_master_df.to_sql("product_master", engine, if_exists="append", index=False)
sales_transactions_df.to_sql("sales_transactions", engine, if_exists="append", index=False)
customer_analytics_df.to_sql("customer_analytics", engine, if_exists="append", index=False)
loyalty_transactions_df.to_sql("loyalty_transactions", engine, if_exists="append", index=False)

print("✅ ETL pipeline finished. retail.db created!")


**sqlite database connections**

In [ ]:
import sqlite3
conn = sqlite3.connect("retail.db")
cursor = conn.cursor()


In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()


Customer_master table

In [ ]:
cursor.execute("PRAGMA table_info(customer_master);")
rows = cursor.fetchall()

for r in rows:
    print(r)


In [ ]:
import pandas as pd

df = pd.read_sql_query("SELECT * FROM sales_transactions LIMIT 10;", conn)
df


product_master

In [ ]:
cursor.execute("PRAGMA table_info(product_master);")
rows = cursor.fetchall()

for r in rows:
    print(r)


In [ ]:
import pandas as pd

df = pd.read_sql_query("SELECT * FROM product_master LIMIT 10 ;", conn)
df


sales_transactions

In [ ]:
cursor.execute("PRAGMA table_info(sales_transactions);")
rows = cursor.fetchall()

for r in rows:
    print(r)


In [ ]:
import pandas as pd

df = pd.read_sql_query("SELECT * FROM sales_transactions LIMIT 10 ;", conn)
df


customer_analytics

In [ ]:
cursor.execute("PRAGMA table_info(customer_analytics);")
rows = cursor.fetchall()

for r in rows:
    print(r)


In [ ]:
import pandas as pd

df = pd.read_sql_query("SELECT * FROM customer_analytics LIMIT 10 ;", conn)
df


loyalty_transaction

In [ ]:
cursor.execute("PRAGMA table_info(loyalty_transactions);")
rows = cursor.fetchall()

for r in rows:
    print(r)


In [ ]:
import pandas as pd

df = pd.read_sql_query("SELECT * FROM loyalty_transactions LIMIT 10 ;", conn)
df
